# VectorDB, Hugging Face & Ollama – Assignment Solutions


## Question 1 Answer
Detailed explanation of VectorDB vs Traditional DBs.

## Question 2 Answer
Types of VectorDBs and their use cases.

## Question 3 Answer
Why ChromaDB is important and its features.

## Question 4 Answer
Benefits of Hugging Face Hub.

## Question 5 Answer
Process & advantages of using pre-trained models.

In [3]:
import sys
!{sys.executable} -m pip install chromadb

  Using cached chromadb-1.4.1-cp39-abi3-win_amd64.whl.metadata (7.3 kB)
  Using cached build-1.4.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached uvicorn-0.40.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
   ---------------------------------------- 0.0/21.4 MB ? eta -:--:--
   -- ------------------------------------- 1.6/21.4 MB 8.8 MB/s eta 0:00:03
   ----- ---------------------------------- 3.1/21.4 MB 8.1 MB/s eta 0:00:03
   --------- ------------------------------ 5.0/21.4 MB 8.4 MB/s eta 0:00:02
   ------------ --------------------------- 6.8/21.4 MB 8.6 MB/s eta 0:00:02
   ---------------- ----------------------- 8.9/21.4 MB 8.7 MB/s eta 0:00:02
   -------------------- ------------------- 10.7/21.4 MB 8.8 MB/s eta 0:00:02
   ---------------------- ----------------- 12.1/21.4 MB 8.7 MB/s eta 0:00:02
   ------------------------- -------------- 13.9/21.4 MB 8.3 MB


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#Question 06
import chromadb

# Create client (new architecture)
client = chromadb.Client()

# Create collection
collection = client.create_collection(name="sample_collection")

# Add documents (Chroma will auto-handle embeddings if not provided)
collection.add(
    ids=["1", "2", "3"],
    documents=[
        "Apple is a fruit",
        "Car engines use fuel",
        "Bananas are yellow"
    ]
)

# Query
results = collection.query(
    query_texts=["Tell me about fruits"],
    n_results=2
)

results

C:\Users\hp\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [01:05<00:00, 1.26MiB/s]


{'ids': [['1', '3']],
 'embeddings': None,
 'documents': [['Apple is a fruit', 'Bananas are yellow']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[0.5656521320343018, 1.090145230293274]]}

In [7]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install transformers datasets accelerate torch


   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----------- ---------------------------- 0.5/1.8 MB 2.5 MB/s eta 0:00:01
   ----------------- ---------------------- 0.8/1.8 MB 1.2 MB/s eta 0:00:01
   ----------------------- ---------------- 1.0/1.8 MB 1.2 MB/s eta 0:00:01
   ----------------------------- ---------- 1.3/1.8 MB 1.2 MB/s eta 0:00:01
   ----------------------------------- ---- 1.6/1.8 MB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 1.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1
  Using cached transformers-4.57.6-py3-none-any.whl.metadata (43 kB)
  Using cached datasets-4.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata

In [9]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"


In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


In [12]:
# Question 7: Fine-tuning Hugging Face model
# !pip install transformers datasets accelerate

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

tokenized = dataset.map(tokenize, batched=True)

training_args = TrainingArguments(output_dir="./fine_tuned_gpt2", num_train_epochs=1)
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized["train"])
# trainer.train()  # commented to avoid long runtime

ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
# Question 8: Custom LLM using Ollama
# Commands to run in terminal:
# curl -fsSL https://ollama.com/install.sh | sh
# Create Modelfile and run:
# ollama create myllama -f Modelfile
# ollama run myllama "Explain AI"

In [15]:
#Question 9
import chromadb

# Use the same client type everywhere
client = chromadb.Client()

collection = client.get_or_create_collection("rag_docs")

collection.add(
    ids=["1"],
    documents=["Machine learning allows computers to learn patterns from data."]
)

query = "What is machine learning?"

results = collection.query(
    query_texts=[query],
    n_results=1
)

context = results["documents"][0][0]
context


'Machine learning allows computers to learn patterns from data.'

In [17]:
# Question 10: Health-tech chatbot architecture with HF + VectorDB + Ollama
from transformers import AutoTokenizer, AutoModel
import chromadb
import torch

# Load public embedding model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().tolist()

# Setup ChromaDB
client = chromadb.Client()
collection = client.get_or_create_collection("medical_docs")

# Insert medical data
collection.add(
    ids=["1"],
    documents=["Jaundice is caused by bilirubin buildup due to liver disease or hepatitis."]
)

# Query
query = "What causes jaundice?"
results = collection.query(query_texts=[query], n_results=1)

results


c:\Users\hp\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


{'ids': [['1']],
 'embeddings': None,
 'documents': [['Jaundice is caused by bilirubin buildup due to liver disease or hepatitis.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None]],
 'distances': [[0.36193040013313293]]}